# New Haven, New Hampshire NDVI in Leafmap with `jupyter-xarray-tiler`

To run this notebook, you'll need the dependencies in this repository's `"play"` dependency group.
For most people, installing the right dependencies might look like:

```bash
uv sync --group dev --group test --group play
```

## First, grab some data and do a calculation on it

We want to see that we can display calculated data from an in-memory dataset.

### Get Sentinel 2 data as an Xarray `DataSet`

#### Define a New Haven, New Hampshire bounding box

In [ ]:
from leafmap.plot import bbox_to_gdf

bbox = [-72.99321, 41.23109, -72.85227, 41.37502]
bbox_to_gdf(bbox).explore()

#### Query the Element84 Earth Search STAC catalog for Sentinel2 data

In [ ]:
from pystac_client import Client

sentinel2_stac_items = (
    Client.open("https://earth-search.aws.element84.com/v1")
    .search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime="2024-06-01/2024-09-01",
        query={"eo:cloud_cover": {"lt": 20}},
    )
    .item_collection()
)

#### Load the data into an Xarray `Dataset`

In [ ]:
import odc.stac

sentinel2_dataset = odc.stac.load(
    sentinel2_stac_items,
    bands=["red", "green", "blue", "nir08"],
    bbox=bbox,
    resolution=10,
    groupby="solar_day",
    chunks={},
)
sentinel2_dataset

### Visualize raw data in RGB

In [ ]:
rgb = sentinel2_dataset[["red", "green", "blue"]].to_array(dim="band").median("time")
rgb_scaled = (rgb / 3000).clip(0, 1)  # Scale and clip for display
rgb_scaled.plot.imshow()

### Calculate NDVI

In [ ]:
ndvi = (
    (
        (sentinel2_dataset.nir08 - sentinel2_dataset.red)
        / (sentinel2_dataset.red + sentinel2_dataset.nir08)
    )
    .median(
        "time",
        keep_attrs=True,
    )
    .where(lambda ndvi: ndvi < 1)
    .compute()
)

ndvi.plot.imshow();

## Test out `jupyter-xarray-tiler`

...with the `ndvi` `DataArray` we just calculated!

In [ ]:
type(ndvi)

In [ ]:
from jupyter_xarray_tiler import TiTilerServer

url = await TiTilerServer().add_data_array(ndvi)

In [ ]:
import leafmap

m = leafmap.Map(center=[41.321482, -72.932739], zoom=10)
m

In [ ]:
m.add_tile_layer(
    url=f"{url}&rescale=0,1",
    name="NH NDVI",
    attribution="Sentinel 2",
)